In [1]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np
import os
import json

resources_path = os.getcwd() + '\\resources\\'
modes_path = resources_path + 'Models\\'

In [3]:
# 模型加载
model_name = "moka-ai/m3e-base"
model = SentenceTransformer(model_name)
model.save(modes_path + "\\m3e-base\\")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\pengx\.conda\envs\RAG\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pengx\.cache\huggingface\hub\models--moka-ai--m3e-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
# 提取文本
questions = json.load(open(resources_path + "questions.json", "r", encoding="utf-8"))
pdf_content = json.load(open(resources_path + "pdf_content.json", "r", encoding="utf-8"))
question_sentences = [x['question'] for x in questions]
pdf_content_sentences = [x['content'] for x in pdf_content]

In [3]:
# 加载模型
model = SentenceTransformer(modes_path + "m3e-base")

# 计算嵌入向量（带进度条）
question_embeddings = model.encode(question_sentences, normalize_embeddings=True, show_progress_bar=True)
pdf_embeddings = model.encode(pdf_content_sentences, normalize_embeddings=True, show_progress_bar=True)

# 转换为numpy数组
question_embeddings = np.array(question_embeddings)
pdf_embeddings = np.array(pdf_embeddings)

# 计算相似度
for query_idx, feat in enumerate(question_embeddings):
    score = feat @ pdf_embeddings.T
    max_score_page_idx = score.argsort()[-1] + 1
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

# 保存答案
with open(resources_path + 'answer.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

In [6]:
question_embeddings.shape

(301, 768)

In [4]:
pdf_embeddings.shape

(354, 768)